In [ ]:
import wandb

from mylingua2tower.utils.mind import get_mind_train, get_mind_val, get_hprarams
from mylingua2tower.iterator import MINDIterator
from mylingua2tower.nrms import NRMSModel

In [ ]:
MIND_type = 'demo'
seed = 42

from src.utils.paths import get_mind_dir, get_glove_dir

mind_data_dir = get_mind_dir()
glove_dir = get_glove_dir()

In [ ]:
wandb.login()

In [ ]:
def main(wandb_config):
    run = wandb.init(project='mylingua2towernrms')
    wandb_config = wandb.config
    
    hparams = get_hprarams(
        MIND_type,
        head_num=16,
        head_dim=16,
        his_size=10,
        seed=seed,
        word_emb_dim=wandb_config.word_emb_dim,
        learning_rate=0.0001,
        mind_data_dir=mind_data_dir,
        glove_dir=glove_dir,
    )

    t_news, t_behaviors = get_mind_train(hparams)
    v_news, v_behaviors = get_mind_val(hparams)

    iterator = MINDIterator
    model = NRMSModel(hparams, iterator, seed=seed)
    model.fit(t_news, t_behaviors, v_news, v_behaviors, wandb_log=True)

In [ ]:
sweep_configuration = {
    'method': 'random',
    'name': 'sweep_his_10',
    'metric': {'goal': 'maximize', 'name': 'val_group_auc'},
    'parameters': 
    {
        'word_emb_dim': {'values': [50, 100, 200, 300]},
     }
}

In [ ]:

sweep_id = wandb.sweep(sweep=sweep_configuration, project='mylingua2towernrms')

In [ ]:
wandb.agent(sweep_id, function=main, count=10)